Baixar *dataset*

In [0]:
!wget https://www.dropbox.com/s/1ozzofw3f7thq3g/MilkQA-v2.tar.gz?dl=1
!tar -xzvf MilkQA-v2.tar.gz?dl=
%cd MilkQA-v2/data

/content/drive/My Drive/I.C. Nádia/Datasets/Training/milkqa/data


Pegar as questões e respostas para substuir pela tradução

In [0]:
import os
import re
from pathlib import Path

files = os.listdir(os.curdir)
files = filter(lambda x: "question" in x, files)
numbers = [re.search("(.*)-", x).group(1) for x in files]

unsolved = []
answers = []
questions = []
qa_number = []
for number in numbers:
        print(number)
        question = Path(number + "-question.txt").read_text()
        question = re.split("<.*>", question)
        question = list(filter(None, question))
        if len(question) != 2:
            unsolved.append(number)
            continue
        
        qa_number.append(number)
        questions.append(question[1])

        answer = Path(number + "-answer.txt").read_text()
        answers.append(answer)

AttributeError: ignored

Resolver a questão 13546

In [0]:
string = """sou de dois vizinhos , e dou assitencia técnica para 
 produtores de leite da agricultura familiar desta cidade . E gostria
    de ter algumas informações sobre cana de açucar cal pecuario na
    suplementação de vacas de leite."""
qa_number.append(13546)
questions.append(string)
answer = Path("13546-answer.txt").read_text()
answers.append(answer)

FileNotFoundError: ignored

Resolver questões 13050, 10551 e 1487: sem assunto

In [0]:
for number in [13050,10551,1487]:
        print(number)
        qa_number.append(number)
        question = Path(str(number) + "-question.txt").read_text()
        question = re.sub("<.*>", "", question)
        questions.append(question)

        answer = Path(str(number) + "-answer.txt").read_text()
        answers.append(answer)

13050
10551
1487


Resolver questões 25414, 16762: sem delimitador \<corpo\>

In [0]:
for number in [25414,16762]:
        print(number)
        qa_number.append(number)
        question = Path(str(number) + "-question.txt").read_text()
        question = re.sub("<.*>", "", question)
        question = re.split("\n\n", question)
        question = list(filter(None, question))

        questions.append(question[1])

        answer = Path(str(number) + "-answer.txt").read_text()
        answers.append(answer)

25414
16762


Criar dicionários com id e perguntas

In [0]:
answers_dict = dict(zip(qa_number, answers))
questions_dict = dict(zip(qa_number, questions))

import json
with open('answers_raw.json', 'w') as json_file:
    json.dump(answers_dict, json_file)

with open('questions_raw.json', 'w') as json_file:
    json.dump(questions_dict, json_file)

Carregar dicionário com id e perguntas

In [0]:
%cd data
import json
with open('answers_raw.json') as json_file:
    answers_dict = json.load(json_file)

with open('questions_raw.json') as json_file:
    questions_dict = json.load(json_file)

/content/drive/My Drive/I.C. Nádia/Datasets/Train/milkqa/data


Pegar o dicionário de tradução

In [0]:
%cd ..
#!wget https://github.com/jubs12/mtdnn_port/raw/master/Translation/milkqa-dic.json

/content/drive/My Drive/I.C. Nádia/Datasets/Training/milkqa


Substituir pelas traduções

In [0]:
import json
from copy import copy

with open('milkqa-dic.json') as f:
    dic = json.load(f)

answers_not_translated = copy(list(answers_dict.keys()))

for key in answers_dict.keys():
    try:
        if key in answers_not_translated:
            answer = answers_dict[key]
            answers_dict[key] = dic[answer]
            answers_not_translated.remove(key)
            #print(key)
    except:
        pass

print("Answers not translated: {}".format(answers_not_translated))
with open('answers.json', 'w') as f:
    json.dump(answers_dict,f)

with open('answers_not_translated.json', 'w') as f:
    json.dump(answers_not_translated,f)

questions_not_translated = copy(list(questions_dict.keys()))
for key in questions_dict.keys():
    try:
        if key in questions_not_translated:
            question = questions_dict[key]
            questions_dict[key] = dic[question]
            questions_not_translated.remove(key)
            #print(key)
    except:
        pass

print("Questions not translated: {}".format(questions_not_translated))
with open('questions.json', 'w') as f:
    json.dump(questions_dict,f)

with open('questions_not_translated.json', 'w') as f:
    json.dump(questions_not_translated,f)

Answers not translated: ['12212']
Questions not translated: ['25414', '16762']


Traduzir restantes

In [0]:
import Auxiliar
missing = [answers_dict['12212'], questions_dict['25414'], questions_dict['16762']]
missing_dct = Auxiliar.translate_lst2dct(missing,'en','missing_milkqa-dic.json')

Enviar restantes

In [0]:
#answers_dict['12212'] =  missing_dict[answers_dict['12212']]
#questions_dict['25414'] =  missing_dict[questions_dict['25414']]
#questions_dict['16762'] =  missing_dict[questions_dict['16762']]

with open('questions.json', 'w') as f:
    json.dump(questions_dict,f)

with open('answers.json', 'w') as f:
    json.dump(answers_dict,f)

Atualizar dicionário original

In [0]:
with open('missing_milkqa-dic.json') as f:
    missing = json.load(f)

with open('milkqa-dic.json') as f:
    dic = json.load(f)

dic.update(missing)

with open('milkqa-dic.json', 'w') as f:
    json.dump(dic,f)

Inserir as perguntas traduzidas na tabela

In [0]:
import pandas as pd
import json
import re
import os

names = [f for f in os.listdir(os.curdir) if '-' in f and 'tsv' in f]
hypotheses = ['hypothesis_{}'.format(i) for i in range(1,51)]
header_names = ['id','label'] + hypotheses
output_header = ['id','label','premise'] + hypotheses

with open('questions.json') as f:
    questions_dict = json.load(f)

for key in questions_dict.keys():
    questions_dict[key] = questions_dict[key].strip()
    questions_dict[key] = questions_dict[key].replace('\n', '')

with open('answers.json') as f:
    answers_dict = json.load(f)

for key in answers_dict.keys():
    answers_dict[key] = answers_dict[key].strip()
    answers_dict[key] = answers_dict[key].replace('\n', '')

for name in names:
    table = pd.read_csv(name, header = None, sep = '\t', names = header_names)
    table['premise'] = table['id'].astype(str).map(questions_dict)
    
    for col in hypotheses:
        table[col] = table[col].astype(str).map(answers_dict)

    output_name = re.sub(r'(.*)-(.*?)', r'\1_\2', name)
    print(output_name)
    table.to_csv(output_name, sep = '\t', index = False, columns = output_header, header = None)

milkqa_test.tsv
milkqa_dev.tsv
milkqa_train.tsv


Tirar formatações html

In [0]:
!sh ftfy_milkqa.sh